In [1]:
from pathlib import Path
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import csv

In [2]:
data_folder = Path('/Users/papo/Sync/MRdata/IoN_Piglet/MRS_RAY')
# data_folder = Path('/Users/papo/Downloads/LWP739_D4')
basis_folder = Path('/Users/papo/Sync/Projects/PAINT_MRS_CSI/3_0T_basis_threonine_no_MM')

sdat_files = [f for f in sorted(data_folder.rglob('*')) if ("act.sdat" in f.name.lower() and "csi" in f.name.lower())]
for file in tqdm(sdat_files):
    ref = Path(f'{str(file)[0:-8]}ref.SDAT')
    csv = Path(f'{str(file)[0:-5]}_both.csv')
    pdf = Path(f'{str(file)[0:-4]}pdf')
    if not csv.is_file():
        # command = f'tarquin --rows 3 --cols 2 --input {file} --output_pdf {pdf} --output_csv {csv} --input_w {ref} --basis_csv {basis_folder}'
        command = f'tarquin --rows 3 --cols 2 --input {file} --output_csv {csv} --input_w {ref} --basis_csv {basis_folder}'
        print(f'{command}\n') 
        os.system(command)


  0%|          | 0/22 [00:00<?, ?it/s]

tarquin --rows 3 --cols 2 --input /Users/papo/Sync/MRdata/IoN_Piglet/MRS_RAY/LWP758/CSI/C2480940_WIP_CSI_PB_auto_TR2S_SENSE_7_2_raw_act.SDAT --output_csv /Users/papo/Sync/MRdata/IoN_Piglet/MRS_RAY/LWP758/CSI/C2480940_WIP_CSI_PB_auto_TR2S_SENSE_7_2_raw_act_both.csv --input_w /Users/papo/Sync/MRdata/IoN_Piglet/MRS_RAY/LWP758/CSI/C2480940_WIP_CSI_PB_auto_TR2S_SENSE_7_2_raw_ref.SDAT --basis_csv /Users/papo/Sync/Projects/PAINT_MRS_CSI/3_0T_basis_threonine_no_MM



100%|██████████| 22/22 [00:17<00:00,  1.25it/s]


In [10]:
def pull_from_csv(csv_file, row=3, header=1):
        lwp_ind = str(csv_file).lower().find('lwp')
        lwp = str(csv_file)[lwp_ind:lwp_ind+6]
        df = pd.read_csv(csv_file, header=[header], error_bad_lines=False)
        try:
            return [lwp,*df.iloc[row].name]
        except:
            return [lwp,*df.iloc[row]]


csv_files = [f for f in sorted(data_folder.rglob('*'), key=lambda x:x.parent.parent.stem[4:6]) if ("act_both.csv" in f.name.lower() and "csi" in f.name.lower())]




rows_to_process = {'BGT': 5, #(BGT = Basal Ganglia and Thalamic, ie deep GM)
                   'WM': 3,
                   }

for label,row_num in rows_to_process.items():
    # print(f'{label=}')
    # print(f'{row_num=}')
    data_list = []
    for file in csv_files:
        data_list.append([*pull_from_csv(file,row=row_num),*pull_from_csv(file,row=row_num+8),*pull_from_csv(file,header=16,row=row_num+1)])
    my_columns = list(pd.read_csv(file, header=[1], error_bad_lines=False))
    my_errors = [item+' Error (SD)' for item in my_columns]
    my_diagnostics = pull_from_csv(file,row=0, header=16)    
    df = pd.DataFrame(data_list, columns=['LWP', *my_columns, 'Error for LWP', *my_errors,*my_diagnostics ])
    df.to_excel(f'{label}_test.xlsx', index=False)



b'Skipping line 18: expected 25 fields, saw 34\nSkipping line 19: expected 25 fields, saw 34\nSkipping line 20: expected 25 fields, saw 34\nSkipping line 21: expected 25 fields, saw 34\nSkipping line 22: expected 25 fields, saw 34\nSkipping line 23: expected 25 fields, saw 34\nSkipping line 24: expected 25 fields, saw 34\n'
b'Skipping line 18: expected 25 fields, saw 34\nSkipping line 19: expected 25 fields, saw 34\nSkipping line 20: expected 25 fields, saw 34\nSkipping line 21: expected 25 fields, saw 34\nSkipping line 22: expected 25 fields, saw 34\nSkipping line 23: expected 25 fields, saw 34\nSkipping line 24: expected 25 fields, saw 34\n'
b'Skipping line 18: expected 25 fields, saw 34\nSkipping line 19: expected 25 fields, saw 34\nSkipping line 20: expected 25 fields, saw 34\nSkipping line 21: expected 25 fields, saw 34\nSkipping line 22: expected 25 fields, saw 34\nSkipping line 23: expected 25 fields, saw 34\nSkipping line 24: expected 25 fields, saw 34\n'
b'Skipping line 18: ex

In [ ]:
# csv_files = [f for f in sorted(data_folder.rglob('*'), key=lambda x:x.parent.parent.stem[4:6]) if ("act.csv" in f.name.lower() and "csi" in f.name.lower())]
# all_ratios = []
# for (i,f) in enumerate(csv_files):
#     print(i,f)
#     print(f.parent.parent.stem)
#     df = pd.read_csv(f, header=[1], error_bad_lines=False)
#     # Lac NAA amplitudes and ratio (loc 3 for voxel 5,5):
#     TNAA = float(df.loc[3]['TNAA'])
#     TLac = float(df.loc[3]['Threonine']) + float(df.loc[3]['Lac'])
    
#     # Lac2NAA error (loc 9 for voxel 5,5):
#     TNAA_err = float(df.loc[9]['TNAA'])
#     TLac_err = np.sqrt( float(df.loc[9]['Lac']) **2 + float(df.loc[9]['Threonine']) **2 ) 
#     try:
#         Lac2NAA = TLac/TNAA
#         Lac2NAA_err = np.sqrt( (TLac_err/TLac) **2 + (TNAA_err/TNAA)**2 ) * Lac2NAA

#     except:
#         Lac2NAA = np.nan
#         Lac2NAA_err = np.nan   
#     print(f'{Lac2NAA=}')
#     print(f'{Lac2NAA_err=}')

#     GM_ratios.append([f.parent.parent.stem, Lac2NAA, Lac2NAA_err])


# all_ratios_df = pd.DataFrame(all_ratios, columns=['LWP', 'Lac2NAA', 'Lac2NAA_error'])


